In [15]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from pathlib import Path

In [16]:
output_notebook()

Loading BokehJS ...

In [18]:
edges_path = Path(r'C:\Users\KSpicer\Documents\GitHub\solo_projects\weird_fiction_visualizations\datasets\fake_data_gephi_file.csv')
edges_df = pd.read_csv(edges_path)
edges_df = edges_df.dropna()
edges_df['target'] = edges_df['target'].astype('int')

G = nx.from_pandas_edgelist(edges_df, source='source', target='target', edge_attr='Semester and Year')

In [21]:
title = 'Weird Fiction Discussion Board Visualization'

HOVER_TOOLTIPS = [('Poster', '@index')]

plot = figure(tooltips=HOVER_TOOLTIPS,
    tools='pan, wheel_zoom, save, reset', active_scroll='wheel_zoom',
    x_range = Range1d(-50, 50), y_range = Range1d(-50, 50), title=title)

network_graph = from_networkx(G, nx.kamada_kawai_layout, scale=50, center=(0,0))

network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)
show(plot)

In [29]:
degrees = dict(nx.degree(G))
nx.set_node_attributes(G, name='degree', values=degrees)
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

color_palette = Oranges8

HOVER_TOOLTIPS_1 = [('Poster', '@index'), ('Degree', '@degree')]
plot_1 = figure(tooltips=HOVER_TOOLTIPS_1,
    tools='pan, wheel_zoom, save, reset', active_scroll='wheel_zoom',
    x_range = Range1d(-50, 50), y_range = Range1d(-50, 50), title=title)

network_graph = from_networkx(G, nx.circular_layout, scale=50, center=(0,0))
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot_1.renderers.append(network_graph)

show(plot_1)